# Урок 2. Переобучение на примере линейной регрессии


Давайте проведём эксперимент и увидим на примере, как переобучается линейная регрессия. Для начала загрузим датасет из csv

In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv('data/non_linear.csv', sep=',')
data.head()

x_train   y_train
0  0.138368  0.838812
1  0.157237  0.889313
2  0.188684  1.430040
3  0.685553  1.717309
4  0.874237  2.032588

Тренироваться будем на полиномиальной регрессии, потому что она сильно склонна к переобучению и мы уже знакомы с этой моделью. Сгенерируем полиномиальные признаки так же, как вы уже делали в модуле "Линейная регрессия. Часть I".

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

def generate_degrees(source_data: list, degree: int):
    """Функция, которая принимает на вход одномерный массив, а возвращает n-мерный
    Для каждой степени от 1 до  degree возводим x в эту степень
    """
    return np.array([
          source_data**n for n in range(1, degree + 1)  
    ]).T

Обучаем модель на валидации, проверяем на контроле для степени полинома *degree=8*. Для разбиения мы воспользуемся функцией train_test_split (ссылка на доку: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Функция принимает следующие аргументы:

* $X$ и $y$ - массивы, которые хотим расщепить на валидацию и контроль
* *test_size* принимает значения от нуля до единицы и означает долю объектов, которые нужно отложить на валидацию (обычно выбирают значения между $0.15$ и $0.35$)
* *random_state* - любое целое число. Функция разбивает выборку случайным образом, но если вы задали параметр random_state, то разбиение не будет меняться в разных запусках программы. Это нужно для воспроизводимости исследований и является признаком хорошего тона в программировании.

In [4]:
degree = 8
X = generate_degrees(data['x_train'], degree)
y = data.y_train.values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print("Качество на валидации: %.3f" % mean_squared_error(y_valid, y_pred))
print("Качество на обучении: %.3f" % mean_squared_error(y_train, y_pred_train))

Качество на валидации: 0.119
Качество на обучении: 0.052


Теперь обучим полиномиальную регрессию для степени *degree = 12* c параметром регуляризации *alpha=0*

In [5]:
degree = 12
X = generate_degrees(data['x_train'], degree)
y = data.y_train.values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print("Качество на валидации: %.3f" % mean_squared_error(y_valid, y_pred))
print("Качество на обучении: %.3f" % mean_squared_error(y_train, y_pred_train))

Качество на валидации: 0.125
Качество на обучении: 0.051


Как изменилась ошибка на обучении? Было $0.052$, стало $0.051$, т.е. *ошибка на обучении падает*.

Как изменилась ошибка на валидации? Было $0.119$, стало $0.125$, т.е. *ошибка на валидации растёт*.

Это же верный **признак переобучения**! Ошибка на валидации растёт, а на обучении падает. Степень полинома $n=12$ хуже, чем степень полинома $n=8$, модель переобучилась. Это печально, как же нам победить переобучение?

**Практическое задание**: предлагаю вам победить переобучение самостоятельно! Обучите *Ridge* регрессию с параметром регуляризации $\alpha=0.01$. 

Как изменилась ошибка на обучении? Как изменилась ошибка на валидации? Удалось ли победить переобучение?

In [8]:
# -- ВАШ КОД ТУТ --
degree = 12
X = generate_degrees(data['x_train'], degree)
y = data.y_train.values
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)
model = Ridge(alpha=0.01).fit(X_train, y_train)
y_pred = model.predict(X_valid)
y_pred_train = model.predict(X_train)
print("Качество на валидации: %.3f" % mean_squared_error(y_valid, y_pred))
print("Качество на обучении: %.3f" % mean_squared_error(y_train, y_pred_train))

Качество на валидации: 0.077
Качество на обучении: 0.058


Я надеюсь, что переобучение с помощью регуляризации победить вам удалось!

Чему полезному мы научились?

* узнали, что такое переобучение
* научились детектировать его с помощью валидационной выборки
* смогли победить переобучение с помощью библиотечного класса Ridge